# Draft

# Statistical Machine Translation Using IBM Translation Models

An ongoing project to translate English to French.

## Introduction

The IBM translation models are a family of statistical machine translation algorithms that date to the late 1980s. The models were created as part of IBM's "Candide" project, which aimed to automatically translate French to English. As an excercise in theory, I decided to try implementing the models myself. The goal of this implementation is efficiency: I want to build a learning algorithm that can be trained even on home computers.


The IBM models work by estimating the conditional probability that a given sentence in one language is the translation of a given  sentence in the other. Learning a distribution like this is difficult because training data is generally sparse. An algorithm is unlikely to see the same string translated two different ways, and unless one vectorizes strings very cleverly, there is little relationship between the Euclidian distance of two string-vectors and their actual semantic relationship. 

Clever vectorization was not the strategy used by Candide team. Rather, they chose to make strict assumptions about the range of distributions the translation model would consider. With these restrictions in place, they then chose the likeliest such distribution.

However, the conditional distributions considered by the Candide team were too simple to adequately discriminate against ill-formed strings. While the likeliest distribution generally assigned high conditional probabilities to correct translations, it would also assign high probabilities to many ungrammatical strings with the right words in approximately the right order.

For example, the simplest IBM model ignores word order entirely, considering the English strings *I think therefore I am* and *am I I therefore think* to be equally likely translations of the French string *je pense donc je suis*. Though each IBM model is better than the last, none of them adequately solve the problem of grammaticality.

The Candide team solved this problem by adopting a "noisy channel" approach to translation. Rather than directly estimate the probability

$$\mathbb{P}(\mathbf{e}|\mathbf{f})$$

that an English string $\mathbf{e}$ is the translation of a French string $\mathbf{f}$, they chose instead to estimate the product

$$\mathbb{P}(\mathbf{e})\mathbb{P}(\mathbf{f}|\mathbf{e}).$$

Recall that per Bayes' Theorem, we have

$$\mathbb{P}(\mathbf{e}|\mathbf{f}) = \frac{\mathbb{P}(\mathbf{e})\mathbb{P}(\mathbf{f}|\mathbf{e})}{\mathbb{P}(\mathbf{f})}.$$

Therefore, the English string $\mathbf{e}$ that maximizes $\mathbb{P}(\mathbf{e}|\mathbf{f})$ is the same English string maximizing $\mathbb{P}(\mathbf{e})\mathbb{P}(\mathbf{f}|\mathbf{e}).$

In a 1993 article, the Candide team playfully described their reasoning as follows:
>A string of English words, $\mathbf{e}$, can be translated into a string of French words in many different ways. Often, knowing the broader context in which $\mathbf{e}$ occurs may serve to winnow the field of acceptable French translations, but even so, many acceptable translations will remain; the choice among them is largely a matter of taste. In statistical translation, we take the view than every French string, $\mathbf{f}$, is a possible translation of $\mathbf{e}$. We assign to every pair of strings $(\mathbf{e}, \mathbf{f})$ a number $\mathbb{P}(\mathbf{e}|\mathbf{f})$, which we interpret as the probability that a translator, when presented with $\mathbf{e}$, will produce $\mathbf{f}$ as his translation. We further take the view that when a native speaker of French produces a string of French words, he has actually concieved of a string of English words, which he translated mentally. Given a French string $\mathbf{f}$, the job of our translation sustem is to find the string $\mathbf{e}$ that the native speaker had in mind then he produced $\mathbf{f}$. We minimize our chance of error by choosing that string $\hat{\mathbf{e}}$ for which $\mathbb{P}(\mathbf{e} | \mathbf{f})$ is greatest. (Brown et. al. 1993)

Practically, the noisy channel approach allows one to shift responsibility for producing well-formed English strings onto the distribution $\mathbb{P}(\mathbf{e})$. That distribution is estimated by an English "language model", which is chosen to assign very low probabilites to ill-formed strings. This frees up the "translation model", which estimates $\mathbb{P}(\mathbf{f} | \mathbf{e})$, to focus on choosing the right words in the right number in approximately the right order. The IBM models that I implement as part of this project are strictly "translaton models", and they should be evaluated with this strategy in mind.

## IBM Model 1

My implementation of IBM's simplest translation model is contained in the notebook "IBM Model 1.ipynb".

### Model 1 Independence Assumptions

Model 1 assumes that the probability $\mathbb{P}(\mathbf{f} | \mathbf{e})$ is a conditional distribution of the form

$$\mathbb{P}(\mathbf{f} | \mathbf{e}) = \epsilon(m|l)\prod_{f \in \mathcal{F}} \left(\sum_{e \in \mathcal{E}} c(e, \mathbf{e})t(f|e)\right)^{c(f, \mathbf{f})},$$

where
- $\mathcal{E}$ is the set of all English words,
- $\mathcal{F}$ is the set of all French words, 
- the function $c(w, \mathbf{w})$ counts how many times a word w shows up in a string $\mathbf{w}$
- the conditional distribution $t(f|e)$ estimates the probability that an English word $e$ translates to a French word $f$, and
- the conditional distribution $\epsilon(m|l)$ estimates the probability that an English string of length $l$ translates to a French string of length $m$.

This model can be justified by the following assumptions. We assume that the probability that an English string $\mathbf{e}$ translates to a French string containing the French word $f$ is a conditional distibution of the form 

$$\mathbb{P}(f | \mathbf{e}) = \sum_{e \in \mathcal{E}} c(e, \mathbf{e})t(f|e).$$

We then imagine that translation of $\mathbf{e}$ into $\mathbf{f}$ is a sequence of independent trials. First, a length $m$ is chosen according to the conditional distribution $\epsilon(m|l)$. Next, $m$ french words are independently drawn one-by-one per the conditional distribution $\mathbb{P}(f | \mathbf{e}).$

Every distribution $\mathbb{P}(\mathbf{f}|\mathbf{e})$ of this form is uniquely determined by the distributions $\epsilon$ and $t$. Our choice of $\epsilon$ and $t$ is informed by a sample $S$ we draw of $n$ translated strings $(\mathbf{e}, \mathbf{f})$. Given such a sample, we choose $\epsilon$ and $t$ to maximize the cross-entropy function

$$\frac{1}{n} \sum_{(\mathbf{e}, \mathbf{f}) \in S}\ln(\mathbb{P}(\mathbf{f}|\mathbf{e})) = \frac{1}{n} \sum_{(\mathbf{e}, \mathbf{f}) \in S} \left( \ln(\epsilon(m|l)) + \sum_{f \in \mathcal{F}}c(f, \mathbf{f})\ln \left( \sum_{e \in \mathcal{E}} c(e, \mathbf{e})t(f|e) \right)\right) .$$

### Cross Entropy

For those unfamiliar with cross-entropy, this may seem an odd thing to maximize. We justify here it as follows. We can define a distribution $\hat{\mu}$ over english strings with the formula

$$\hat{\mu}(\mathbf{e}) = \frac{c(\mathbf{e}, S)}{n},$$

where $c(\mathbf{e}, S)$ is the number of times the string $\mathbf{e}$ appears in the sample $S$. We will call $\hat{\mu}$ the empirical marginal distribution of our sample $S$, and we will think of it as a very simple English language model. It assigns low probabilities to English strings that appear in $S$, and it assigns no probability at all to strings that do not appear in $S$.

Similarly, we can define a conditional distribution $\hat{K}$ from the set of all English strings to the set of all French strings by

$$ \hat{K}(\mathbf{f} | \mathbf{e}) = \frac{c((\mathbf{e}, \mathbf{f}), S)}{c(\mathbf{e}, S)},$$ 

where $c((\mathbf{e}, \mathbf{f}), S)$ is the number of times the pair $(\mathbf{e}, \mathbf{f})$ appears in the sample $S$. We wll call $\hat{K}$ the empirical kernel of $S$, and we will think of $\hat{K}$ as a very simple English-to-French translation model.

Though $\hat{K}$ is a comically simple model, it is, in fact, all that our sample tells us about the relationship between the two languages. We will choose the parameters $t$ and $\epsilon$ so that the resulting conditional distribution is as close as possible to the empirical kernel.

The measure of distance we use is Kullback-Liebler divergence. KL divergence is convex, positive-definite function of two distributions. It is not symmetric, and it does not satsify the triangle inequality, so it is not a metric but merely a "divergence". However, it does upper bound the total variation metric per Pinsker's inequality. 

Given two distributions $\mu$, $\nu$ over some set, with $\mu$ absolutely continuous with respect to $\nu$, the KL divergence $D(\mu || \nu)$ is given by

$$D(\mu || \nu) = \int \mathrm{d}\mu \ln\left( \frac{\mathrm{d}\mu}{\mathrm{d}\nu} \right),$$

where $\frac{\mathrm{d}\mu}{\mathrm{d}\nu}$ is the Radon-Nikodym derivative of $\mu$ with respect to $\nu$. In our case, we seek $t$ and $\epsilon$ that minimize the divergence 

$$D(\hat{K} \left( \cdot | \mathbf{e}) || \mathbb{P}(\cdot | \mathbf{e}) \right)$$ 

for all $\mathbf{e}$. Of course, we should prioritize minimizing this expression over strings $\mathbf{e}$ that are well formed. For this reason, we weight each divergence $D(\hat{K} \left( \cdot | \mathbf{e}) || \mathbb{P}(\cdot | \mathbf{e}) \right)$ in our loss function by the probability that the string $\mathbf{e}$ is well-formed, which we approximate by the empirical marginal distribution $\hat{\mu}$. In short, we choose to mininize the expression

\begin{align}
\int \hat{\mu}(\rm{d}\mathbf{e}) \, D(\hat{K} \left( \cdot | \mathbf{e}) || \mathbb{P}(\cdot | \mathbf{e}) \right) &= \sum_\mathbf{e} \hat{\mu}(\mathbf{e}) \sum_\mathbf{f}\hat{K}(\mathbf{f} | \mathbf{e}) \ln\left( \frac{\hat{K}(\mathbf{f} | \mathbf{e})}{\mathbb{P}(\mathbf{f} | \mathbf{e})} \right) \\
&= \frac{1}{n} \sum_{(\mathbf{e}, \mathbf{f}) \in S}\ln\left( \frac{\hat{K}(\mathbf{f} | \mathbf{e})}{\mathbb{P}(\mathbf{f} | \mathbf{e})} \right),
\end{align}

the minimization of which is equivalent to maximizing the cross-entropy shown above.

### EM Algorithm

The careful reader has already noticed that when maximizing the cross-entropy over $t$ and $\epsilon$, the two variables can be solved seperately.